 # Study of road traffic data

 ## Preliminaries

 ### The Dataset

 Outlined here are the what each of the 31 columns represent

 * Light_Conditions: The illumination conditions prevalent at the site where the accident occurred.

 * Road_Type: The classification describing the type of road where the accident occurred, which can include roundabouts, one-way streets, dual carriageways, or single carriageways.

 * Date: The calendar date when the accident occurred.

 * 2nd_Road_Class: A classification indicating the category of the secondary road involved in the accident.

 * 1st_Road_Class: A classification signifying the category of the primary road involved in the accident, encompassing highways, A roads, B roads, or other classifications.

 * Road_Surface_Conditions: The state of the road surface, encompassing factors like wet, icy, or dry conditions.

 * Location_Easting_OSGR: The eastward positional coordinate of the accident site, referenced to the Ordnance Survey Grid.

 * Urban_or_Rural_Area: An indication of whether the accident transpired in an urban or rural region.

 * Speed_limit: The legal speed limit imposed on the road segment where the accident transpired.

 * Police_Force: The law enforcement agency responsible for responding to and managing the accident.

 * Location_Northing_OSGR: The northward positional coordinate of the accident site, referenced to the Ordnance Survey Grid.

 * Number_of_Vehicles: The count of automobiles involved in the accident.

 * Day_of_Week: The specific day of the week when the accident took place, represented numerically (1 for Sunday, 2 for Monday, etc.).

 * Weather_Conditions: The atmospheric conditions, such as rain, snow, or fog, prevailing at the accident location.

 * 2nd_Road_Number: The numerical identifier assigned to the secondary road involved in the accident.

 * Pedestrian_Crossing-Human_Control: The type of human control employed at pedestrian crosswalks located at or near the accident scene.

 * Junction_Control: The mode of control governing the junction or intersection where the accident transpired.

 * Longitude: The horizontal geographical coordinate pinpointing the accident's position.

 * Number_of_Casualties: The tally of individuals who sustained injuries or fatalities as a result of the accident.

 * Pedestrian_Crossing-Physical_Facilities: The type of physical infrastructure available at pedestrian crosswalks situated at or near the accident location.

 * Local_Authority_(District): The governing body overseeing the administrative district where the accident transpired.

 * Special_Conditions_at_Site: Unusual or specific circumstances present at the accident site.

 * 1st_Road_Number: The numerical identifier assigned to the primary road involved in the accident.

 * Junction_Detail: A detailed description of the junction or intersection at which the accident occurred.

 * Latitude: The vertical geographical coordinate pinpointing the accident's position.

 * Local_Authority_(Highway): The governing body responsible for maintaining and managing the road on which the accident occurred.

 * Did_Police_Officer_Attend_Scene_of_Accident: A binary indicator revealing whether law enforcement personnel attended the accident scene.

 * Accident_Index: A unique identifier assigned to each accident incident.

 * Accident_Severity: A measure indicating the extent of seriousness of the accident, reflecting its impact and outcomes.

 * Time: The precise moment when the accident took place.

 * Carriageway_Hazards: Hazards or dangers on the roadway at the accident location.



 ## Exploring the data

 Before proceeding, our initial step involves exploring the structure and assessing the overall data quality of our dataset

In [ ]:
import pandas as pd
import geopandas as gpd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
from libpysal.weights import Kernel
from esda.moran import Moran
from esda.moran import Moran_Local
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import chi2_contingency
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
df = pd.concat([ pd.read_csv("/Users/bashir/Documents/UCL/Quantitative Methods/Written Investigation/Datafiles/accidents_2005_to_2007.csv") ,
                         pd.read_csv("/Users/bashir/Documents/UCL/Quantitative Methods/Written Investigation/Datafiles/accidents_2009_to_2011.csv"), 
                         pd.read_csv("/Users/bashir/Documents/UCL/Quantitative Methods/Written Investigation/Datafiles/accidents_2012_to_2014.csv")], axis=0, ignore_index=True)


df=df.reset_index(drop=True)
df


In [ ]:
df.info()


In [ ]:
df.describe()


 ---------

 One of the initial observations I made is that the Index column appears to contain duplicate values. This may pose a potential issue for us down the line, as we require a unique identifier to serve as an index.

In [ ]:
for i in ["Accident_Index",["Longitude","Latitude"],"Latitude","Longitude","Date","Time",["Date","Time"],["Date","Time","Latitude","Longitude"]]:
    print(i,' ',len(df.drop_duplicates(subset=i)))

print(len(df))


 ----

In [ ]:
def coefficient_of_variation(df):
    cv_data = {}
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            col_data = df[col].dropna()  
            if len(col_data) > 0:  
                col_cv = col_data.std() / col_data.mean() 
                cv_data[col] = col_cv

    cv_df = pd.DataFrame(cv_data, index=['coefficient_of_variation']).T
    return cv_df

coefficient_of_variation_df = coefficient_of_variation(df)

plt.figure(figsize=(12, 8))
sns.heatmap(coefficient_of_variation_df, annot=True, cmap="YlGnBu")
plt.title("Heatmap of Coefficient of Variation")
plt.show()

 * 'Number_of_Vehicles' has a moderate CV of 0.39, indicating a moderate level of spread or variability in the data.

 * 'Number_of_Casualties' exhibits a higher CV of 0.611, indicating that the values are more widely distributed compared to 'Number_of_Vehicles.'

 * 'Longitude' has a negative CV of -0.973, which is unusual. This suggests that the standard deviation of longitude values is greater than the mean. This could be attributed to the fact that longitude values are typically negative, causing the mean to be negative, while the standard deviation remains positive.

 * 'Latitude' shows a very low CV of 0.028, indicating that the values are very close to the mean.

 These CV findings provide valuable insights into the dispersion of data in our dataset, which, in turn, can inform our data cleaning and structuring efforts as we move forward with our analysis

 ## Data Cleaning/Processing

In [ ]:
print(df.isna().sum())
print(" ")
print(len(df))


 We can initiate the data cleaning process by creating an initial pipeline for our dataset. In this phase, we'll begin by removing columns that are unrelated to accidents and eliminating null values that are of minor significance (i.e., those with fewer than 180 occurrences).

In [ ]:
def remove_dup(df):
    return df.drop_duplicates(subset=['Date', 'Time', 'Longitude', 'Latitude'], keep='first')

def drop_drow(df):
    return df.dropna(subset=["Time", "Pedestrian_Crossing-Human_Control", "Pedestrian_Crossing-Physical_Facilities", "Weather_Conditions", "Latitude", "Longitude"])

def drop_column(df, columns):
    return df.drop(columns, axis=1)

def fill_na(df, columns, value):
    for column in columns:
        df[column] = df[column].fillna(value)
    return df

def clean_pipineline(df):
    df = remove_dup(df)
    print("Number of rows after removing duplicates: ", len(df))
    df = drop_column(df, ["LSOA_of_Accident_Location", 'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Did_Police_Officer_Attend_Scene_of_Accident'])
    print("Number of rows after dropping columns: ", len(df))
    df = drop_drow(df)
    print("Number of rows after dropping rows: ", len(df))
    
    df = fill_na(df, ['Special_Conditions_at_Site', 'Carriageway_Hazards'], 'No information recorded')
    print("Number of rows after filling NaN values: ", len(df))

    return df

df = clean_pipineline(df)



In [ ]:
print(df.isna().sum())
print(" ")
print(len(df))


 -------

 Let's now conduct a more detailed examination of the column contents. To facilitate our data manipulation efforts, we will proceed by creating dedicated functions for these columns. For the more complex categorical columns, we will address missing values by using a Decision Tree model for imputation.


In [ ]:
def fill_nulls_with_decision_tree(df, feature_cols, target_col):
        
    train_df = df[df[target_col].notnull()]
    test_df = df[df[target_col].isnull()]

    model = DecisionTreeClassifier(random_state=42)    
    model.fit(train_df[feature_cols], train_df[target_col])
    imputed_values = model.predict(test_df[feature_cols])
    df.loc[df[target_col].isnull(), target_col] = imputed_values

    return df,model

def dummify(df, columns):
    df_dummified = pd.get_dummies(df, columns=columns)
    return df_dummified

def undummify(df_dummified, column_name):
    column_names = df_dummified.columns.tolist()
    dummy_columns = [col for col in column_names if column_name in col]
    df_undummified = df_dummified.copy()
    df_undummified[column_name] = df_dummified[dummy_columns].idxmax(axis=1).str.replace(f"^{column_name}_", "")
    df_undummified.drop(dummy_columns, axis=1, inplace=True)
    return df_undummified

def create_temporary_mapper(df, column_name):
    unique_values = df[column_name].unique()
    new_values = range(len(unique_values))
    mapper = dict(zip(unique_values, new_values))
    return mapper

def display_aggregation(df, *columns):
    H = list(columns)
    C= list(columns)
    C.append('Accident_Index')
    print(C)
    print(df[C].groupby(by=H).count().sort_values(by=[H[0]]+["Accident_Index"],ascending=[True,False]).to_string())



In [ ]:
df = df[df['Date'].str.match(r'^\d{2}/\d{2}/\d{4}$')]
df = df[df['Time'].str.match(r'^\d{2}:\d{2}$')]

df["int-Date"] = df["Date"].apply(lambda x: int(x[3:5]))
df["int-Time"] = df["Time"].apply(lambda x: int(str(x)[0:2]))



 -----

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(data=df, x="Light_Conditions", palette="viridis")
plt.show()




 We have observed that we can transform this variable into an ordinal category, representing the degree of darkness as follows: 0 for No presence of darkness, 1 for Presence with light, and 2 for Total darkness.

In [ ]:
df["Darkness_Presence"]=df["Light_Conditions"].replace(['Daylight: Street light present',
       'Darkness: Street lights present and lit',
       'Darkness: Street lighting unknown',
       'Darkness: Street lights present but unlit',
       'Darkeness: No street lighting'] , [0,1,2,2,2])


In [ ]:
df=df.drop(["Light_Conditions"],axis=1)


 ---

 Exploring the action of 'wind'

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(data=df, x="Weather_Conditions", palette="Set1")
plt.show()


In [ ]:
df["Weather_Conditions"]=df["Weather_Conditions"].replace(['Raining without high winds', 'Fine without high winds',
       'Snowing without high winds', 'Other', 'Fine with high winds',
       'Raining with high winds', 'Fog or mist',
       'Snowing with high winds'],["No High winds","No High winds","No High winds","No High winds","High winds","High winds","Fog","High winds"])


In [ ]:
mapping = {
    "No High winds": 0,
    "High winds": 2,
    "Fog": 0,
    "Unknown":0
}

df["Weather_Conditions"] = df["Weather_Conditions"].replace(mapping)


 ----

 Now, we'll address the missing values in the "Road_Surface_Conditions" column by utilizing information from other related columns to predict suitable replacements.

In [ ]:
aggregation = df.groupby("Road_Surface_Conditions").size().reset_index(name="Count")
print(aggregation)


In [ ]:
def create_temporary_mapper(df, column_name):
    unique_values = df[column_name].unique()
    mapper = dict(zip(unique_values, range(len(unique_values))))
    return mapper

road_surface_conditions_mapper = create_temporary_mapper(df, "Road_Surface_Conditions")
print(road_surface_conditions_mapper)


In [ ]:
mapping = {
    'Dry': 5,
    'Wet/Damp': 4,
    'Frost/Ice': 3,
    'Snow': 2,
    'Flood (Over 3cm of water)': 1,
    pd.NA:pd.NA
}

df['Road_Surface_Conditions_Ordinal'] = df['Road_Surface_Conditions'].replace(mapping)


In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Road_Surface_Conditions_Ordinal', palette='Set2')
plt.show()


In [ ]:
variables = ['Urban_or_Rural_Area', 'Accident_Severity', 'Darkness_Presence', 'Weather_Conditions']

fig, axs = plt.subplots(2, 2, figsize=(10, 10))
fig.suptitle('Road Surface Conditions vs. Other Variables')

for i, var in enumerate(variables):
    row = i // 2
    col = i % 2
    
    sns.countplot(data=df, x='Road_Surface_Conditions_Ordinal', hue=var, ax=axs[row, col])
    
    axs[row, col].set_title(var)
    axs[row, col].legend(loc='upper right')

fig.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()



In [ ]:
print("Number_of_Vehicles")
display_aggregation(df,"Road_Surface_Conditions_Ordinal","Number_of_Vehicles")
print(" ")
print("Number_of_Casualties")
display_aggregation(df,"Road_Surface_Conditions_Ordinal","Number_of_Casualties")



 Features of note: : ['Urban_or_Rural_Area', 'Accident_Severity',"Number_of_Vehicles","Number_of_Casualties", 'Weather_Conditions']

In [ ]:
display_aggregation(df,"Road_Surface_Conditions_Ordinal")



In [ ]:
df_cleaned,clf = fill_nulls_with_decision_tree(df, ['Urban_or_Rural_Area', 'Accident_Severity',"Number_of_Vehicles","Number_of_Casualties", 'Weather_Conditions'],'Road_Surface_Conditions_Ordinal')



In [ ]:
display_aggregation(df,"Road_Surface_Conditions_Ordinal")



In [ ]:
df_cleaned['Road_Surface_Conditions']=df_cleaned['Road_Surface_Conditions_Ordinal']


In [ ]:
df_cleaned=df_cleaned.drop(['Road_Surface_Conditions'],axis=1)


 ----

 ### Junction_Control

In [ ]:
display_aggregation(df,"Junction_Control")



In [ ]:
df_cleaned["Junction_Control"]=df_cleaned["Junction_Control"].replace([pd.NA],["No Junction"])


In [ ]:
df_cleaned=df_cleaned.drop(['Junction_Detail'],axis=1)


 -----

 ### Number_of_Vehicles / Number_of_Casualties

In [ ]:
plt.scatter(df_cleaned["Number_of_Vehicles"], df_cleaned["Number_of_Casualties"])
plt.xlabel("Number_of_Vehicles")
plt.ylabel("Number_of_Casualties")
plt.title("Scatter Plot of Number_of_Vehicles vs. Number_of_Casualties")
plt.show()



In [ ]:
plt.boxplot(df_cleaned["Number_of_Vehicles"])
plt.xlabel("Number_of_Vehicles")
plt.title("Box Plot of Number_of_Vehicles")
plt.show()



In [ ]:
plt.boxplot(df_cleaned["Number_of_Casualties"])
plt.xlabel("Number_of_Casualties")
plt.title("Box Plot of Number_of_Casualties")
plt.show()



In [ ]:
df_cleaned=df_cleaned[(df_cleaned["Number_of_Vehicles"]<20)&(df_cleaned["Number_of_Casualties"]<45)]


 -----

 ### Speed_Limit

In [ ]:
speed_limit_counts = df_cleaned['Speed_limit'].value_counts()
labels = speed_limit_counts.index
counts = speed_limit_counts.values

plt.figure(figsize=(10, 11))
plt.pie(counts, labels=None, autopct=None, startangle=140)
plt.title("Pie Chart of Speed Limits")
plt.axis('equal')  

percentage_labels = ['{:.1f}%'.format(pct) for pct in (counts / counts.sum() * 100)]
plt.legend(labels=[f'{label} ({percentage})' for label, percentage in zip(labels, percentage_labels)], title="Speed Limit", loc="upper left")

plt.show()



 -----

 ### "Pedestrian_Crossing-Human_Control" & 'Pedestrian_Crossing-Physical_Facilities'

In [ ]:
custom_palette = sns.color_palette("Set1") 

plt.figure(figsize=(10, 6))
sns.countplot(data=df_cleaned, x="Pedestrian_Crossing-Human_Control", palette=custom_palette)
plt.title("Pedestrian Crossing Human Control")
plt.xlabel("Control Type")
plt.ylabel("Count")

plt.show()



In [ ]:
df_cleaned['Pedestrian_Crossing-Physical_Facilities'] = df_cleaned['Pedestrian_Crossing-Physical_Facilities'].str.replace("-", "\n")

custom_palette = sns.color_palette("Set2")  

plt.figure(figsize=(20, 10))
sns.countplot(
    data=df_cleaned,
    x="Pedestrian_Crossing-Physical_Facilities",
    palette=custom_palette
)
plt.title("Pedestrian Crossing Physical Facilities")
plt.xlabel("Facilities Type")
plt.ylabel("Count")
plt.show()




 As evident, it would be more beneficial to exclude these values, particularly because our primary focus is on the accidents themselves rather than the involvement of pedestrians.







In [ ]:
df_cleaned.groupby(by="Pedestrian_Crossing-Human_Control").size()


In [ ]:
df_cleaned=df_cleaned.drop(["Pedestrian_Crossing-Human_Control",'Pedestrian_Crossing-Physical_Facilities'],axis=1)


 -----

 ### "Year", "Day_of_Week"

In [ ]:
custom_palette = sns.color_palette("Set2")

plt.figure(figsize=(10, 6))
sns.countplot(data=df_cleaned, x="Year", palette=custom_palette)
plt.title("Distribution of Accidents by Year")
plt.xlabel("Year")
plt.ylabel("Count")
plt.show()

We can see the amount of traffic accidents has diminished over the years, displaying a downwards trend with an unexpected increase in year 2012. It is possible that this is due to a larger increase in tourism which will need to be explored seperately



In [ ]:
custom_palette = sns.color_palette("Set2")

plt.figure(figsize=(10, 6))
sns.countplot(data=df_cleaned, x="Day_of_Week")
plt.title("Distribution of Accidents by Day of the Week")
plt.xlabel("Day of the Week")
plt.ylabel("Count")
plt.show()

In [ ]:
display_aggregation(df_cleaned,"Day_of_Week","Accident_Severity")


In [ ]:
df_cleaned=df_cleaned.drop(["Day_of_Week"],axis=1)


In [ ]:
original_columns = set(df.columns)
cleaned_columns = set(df_cleaned.columns)
removed_columns = original_columns - cleaned_columns

print("Columns that have been removed:", removed_columns)

 -----

 ### '2nd_Road_Class'

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(df_cleaned,hue='2nd_Road_Class',x='Accident_Severity')


In [ ]:
plt.figure(figsize=(10, 10))
df_filtered = df_cleaned[~df_cleaned['2nd_Road_Class'].isin([-1, 6])]
sns.countplot(data=df_filtered, x='Accident_Severity', hue='2nd_Road_Class')
plt.show()




 I'm skeptical about the relevance of knowing the classification of the second road in our analysis, as it doesn't appear to significantly vary with the severity of the accidents.

In [ ]:
df_cleaned=df_cleaned.drop(['2nd_Road_Class','2nd_Road_Number'],axis=1)


 -----

 ### "Special_Conditions_at_Site"

In [ ]:
df_cleaned.groupby(by=["Special_Conditions_at_Site"]).size()


 There are quite a few NANs in this section and thus it is best to remove from skewing the results

In [ ]:
plt.figure(figsize=(10, 10))

filtered_df = df_cleaned[(df_cleaned["Special_Conditions_at_Site"] != "None") &
                         (df_cleaned["Special_Conditions_at_Site"] != "No information recorded")]

sns.countplot(data=filtered_df, x='Accident_Severity', hue='Special_Conditions_at_Site')
plt.show()



 We want to retain this column because it's important for us to understand how accidents are influenced by this particular feature.







 -----

In [ ]:
df_cleaned=df_cleaned.drop(['Local_Authority_(District)',
       'Local_Authority_(Highway)','Carriageway_Hazards','Accident_Index','Police_Force',
       '1st_Road_Number'],axis=1)


In [ ]:
df_cleaned['datetime'] = pd.to_datetime(df_cleaned['Date'] + ' ' + df_cleaned['Time'], format='%d/%m/%Y %H:%M')
df_cleaned.drop(["Date", "Time", "int-Time", "int-Date"], axis=1, inplace=True)
df_cleaned['datetime'] = df_cleaned['datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')


 ----

 ### Spatial Informations

In [ ]:
uk_map = gpd.read_file("/Users/bashir/Documents/UCL/Quantitative Methods/GBR_adm/GBR_adm1.shp")

years = [2005, 2006, 2007, 2009, 2010, 2011, 2012, 2013, 2014]

for year in years:
    data = df_cleaned[df_cleaned["Year"] == year]
    
    fig, ax = plt.subplots(figsize=(10, 10))
    uk_map.plot(ax=ax, alpha=0.5)
    plt.scatter(data["Longitude"], data["Latitude"], alpha=0.2, s=3, c='r')
    plt.title(f"Scatter plot for year {year}")

    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()



In [ ]:
fig = px.density_mapbox(
    df_cleaned[(df_cleaned["Year"] >= 2005) & (df_cleaned["Year"] <= 2014)],
    lat='Latitude',
    lon='Longitude',
    mapbox_style="stamen-terrain",
    opacity=0.6 
)

fig.show()

 * Looks like most accidents happen, as could be expected; in and around major cities such as London, Liverpool and the Midlands, and around Newcastle and Middlesbrough in the North East. Meanwhile in Scotland there's a big concentration as well, between Glasgow and Edinburgh. In Wales, most accidents occur near the capital, Cardiff. 
 * The majority of the data is centered around the central and southern regions of the UK.
 * Data clusters are more pronounced in larger urban areas.
 * There is a noticeable decline in accidents between 2005 and 2014.

 ## Clusters


 ### NB :

 I encountered a significant issue due to the size of our dataset, as processing over a million rows would be extremely time-consuming. To address this, I will work with a reduced dataset by sampling more than 100,000 occurrences, aiming for approximately 22,000 samples for each year.







 ### Sub-sampling

In [ ]:
data=df_cleaned
data["datetime"]=pd.to_datetime(data["datetime"])

data["Year"]=data["datetime"].apply(lambda x:x.year)
data["Month"]=data["datetime"].apply(lambda x:x.month)

year_month_groups = data.groupby(['Year', 'Month'])

year_month_counts = year_month_groups.size()

total_count = len(data)
year_month_proportions = year_month_counts / total_count

GROUP_SIZE=100000

sample_sizes = np.round(year_month_proportions * GROUP_SIZE).astype(int)

samples = []
for year_month, group in year_month_groups:
    sample = group.sample(sample_sizes[year_month])
    samples.append(sample)
samples = pd.concat(samples)

expected_counts = year_month_proportions * len(samples)

observed = year_month_counts.values.reshape(-1, 1)
expected = expected_counts.values.reshape(-1, 1)
contingency_table = np.concatenate([observed, expected], axis=1)

chi2_stat, p_val, dof, expected_counts = chi2_contingency(contingency_table)

print("Chi-squared test results:")
print(f"  Chi-squared statistic: {chi2_stat:.2f}")
print(f"  Degrees of freedom: {dof}")
print(f"  p-value: {p_val:.5f}")
if p_val < 0.05:
    print("  The proportions of samples taken from each year-month group are significantly different.")
else:
    print("  The proportions of samples taken from each year-month group are not significantly different.")



In [ ]:
samples


In [ ]:
def wranling_data(samples,columns_to_dummify):
    try:
        samples = dummify(samples.drop(["Unnamed: 0"],axis=1),columns_to_dummify)
    except:
        samples = dummify(samples,columns_to_dummify)
    samples=samples.drop(["datetime"],axis=1)
    return samples


In [ ]:
samples=wranling_data(samples,["Road_Type","Junction_Control","Road_Surface_Conditions_Ordinal","Special_Conditions_at_Site"])


In [ ]:
samples


 ### Applying 1st Clustering Algorithm

In [ ]:
for year in samples["Year"].unique():
    try:
        X = samples[samples["Year"] == year].drop("Clusters",axis=1)
    except:
        X = samples[samples["Year"] == year]
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)
    dbscan = DBSCAN(eps=0.8, min_samples=8)
    clusters = dbscan.fit_predict(X_std)
    clusters = np.where(clusters == -1, np.nan, clusters)
    samples.loc[samples["Year"] == year, "Clusters"] = clusters
    samples["Clusters"].fillna(-1, inplace=True)


In [ ]:
def print_one_feature_mode(df,year,cluster,column):
    print(column,' ',df[(df["Year"]==year) & (df["Clusters"]==cluster)][column].unique())

def print_cluster(df,year,cluster,open_plot):
    for i in df.drop(["Clusters"],axis=1).columns[3:]:
        print_one_feature_mode(df,year,cluster,i)
    if open_plot:
        fig, ax = plt.subplots(figsize=(8,8))
        uk_map.plot(ax=ax, alpha=0.5)
        plt.scatter(samples[(samples["Year"]==year) & (samples["Clusters"]==cluster)]["Longitude"], samples[(samples["Year"]==year) & (samples["Clusters"]==cluster)]["Latitude"], alpha=0.2, s=3, c='r')



In [ ]:
columns_to_dummify=["Road_Type","Junction_Control","Road_Surface_Conditions_Ordinal","Special_Conditions_at_Site"]

for i in columns_to_dummify:
    samples=undummify(samples,i)


In [ ]:
for i in samples["Year"].unique():
    print(" ")
    print("** Year : ",i," **")
    for j in samples[samples["Year"]==i]['Clusters'].value_counts().index[1:4]:
        print("Cluster : ",j)
        print_cluster(samples,i,j,False)
        print(" ")


In [ ]:
px.density_mapbox(X, lat='Latitude', lon='Longitude',mapbox_style="stamen-terrain")


 Concentrations can be seen in London, Birmingham and other visible locations here!

 ### Applying Clustering Algorithm for Special Places

 df_test=df_cleaned[df_cleaned["Special_Conditions_at_Site"]!="None"]
 df_test=wranling_data(df_test,["Road_Type","Junction_Control","Road_Surface_Conditions_Ordinal","Special_Conditions_at_Site"])
 
X=df_test

scaler = StandardScaler()
X_std = scaler.fit_transform(X)
dbscan = DBSCAN(eps=0.8, min_samples=8)
clusters = dbscan.fit_predict(X_std)

 df_test['cluster'] = clusters

 columns_to_dummify=["Road_Type","Junction_Control","Road_Surface_Conditions_Ordinal","Special_Conditions_at_Site"]

 for i in columns_to_dummify:
     df_test=undummify(df_test,i)

 for j in df_test.groupby(by="cluster").size().sort_values(ascending=False).index[1:5]:
     print("** CLUSTER ",j," **")
     for i in df_test.drop(["cluster"],axis=1).columns[3:]:
         print(i,' : ',df_test[df_test["cluster"]==j][i].unique())

 -----

 ### Hot Spot Analysis
 Another interesting method to use would be to look for the correlation between the spatial data, this gives a score that can be interpreted as a "Low" or "High" score and allows to have different areas as well.
 For this, we will use libraries made for spatial data analysis!

In [ ]:
data=gpd.GeoDataFrame(df_cleaned, geometry=gpd.points_from_xy(df_cleaned.Longitude, df_cleaned.Latitude))


In [ ]:
data


In [ ]:
import geopandas as gpd
import numpy as np
from libpysal.weights import Kernel
from esda.moran import Moran
from esda.moran import Moran_Local
X=data[data["Year"]==2014].sample(100)
coords = X['geometry'].apply(lambda p: (p.x, p.y)).tolist()

w = Kernel(coords, bandwidth=1000)

moran = Moran(X['Number_of_Vehicles'], w)
print("Moran's I: ", moran.I)

lisa = Moran_Local(X['Number_of_Vehicles'], w)

clusters = X.loc[(lisa.p_sim < 0.05) & (lisa.q==1)]
print(clusters.head())

spots = X.loc[(lisa.p_sim < 0.05) & (lisa.q==3)]
print(spots.head())



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

X['hot_spot'] = 0
X.loc[clusters.index, 'hot_spot'] = 1
X.loc[spots.index, 'hot_spot'] = -1

fig, ax = plt.subplots(figsize=(10, 10))
uk_map.plot(ax=ax, alpha=0.5)
X.plot(column='hot_spot', cmap=sns.diverging_palette(255, 15, as_cmap=True), ax=ax)
plt.axis('off')
plt.show()
